***In this notebook I take the mean average of the crime index for the entire country, instead of the individual values for different cities.*** 

In [2]:
#Importing the relevant stuff:
# Import pandas
import pandas as pd
# Import get_dataframe function from the sql module
from sql_functions  import get_dataframe
from sql_functions  import get_engine
import sqlalchemy
import psycopg2

Now I take the table for the crime index we already have in our database. I do this by combining SQL and Python (writing a SQL function inside of Python.) This way, I can retrieve it from our schema and get it as a pandas dataframe. 

In [3]:
#note: password, host and username are in a different notebook, which will not be uploaded to github due to gitignore

#define schema
schema = 'bestteamever'

# Import the needed tables from sql server and put it in dataframes

df_crime_index = get_dataframe(f'select * from {schema}.final_european_crime_index;')

In [60]:
schema= "bestteamever"

this_df = get_dataframe(f'select * from {schema}.tableau_final_table')

Making sure that it is what I actually want:

In [14]:
df_crime_index.head(20)

,country,city,crime index
0,United Kingdom,Bradford,71.24
1,United Kingdom,Coventry,68.35
2,France,Nantes,65.70
3,Kazakhstan,Almaty,64.17
4,Italy,Catania,63.51
5,United Kingdom,Birmingham,62.68
6,France,Marseille,62.51
7,Romania,Craiova,60.20
8,Italy,Naples,59.96
9,France,Nice,59.43


In [6]:
df_crime_index.country.values

array(['United Kingdom', 'United Kingdom', 'France', 'Kazakhstan',
       'Italy', 'United Kingdom', 'France', 'Romania', 'Italy', 'France',
       'Belarus', 'Belgium', 'Sweden', 'Greece', 'France',
       'United Kingdom', 'France', 'Greece', 'Spain', 'France', 'France',
       'United Kingdom', 'Ukraine', 'Italy', 'France', 'Portugal',
       'Ireland', 'Belgium', 'Ukraine', 'Russia', 'Italy', 'Ireland',
       'Russia', 'France', 'Italy', 'United Kingdom', 'Romania', 'France',
       'Spain', 'Ukraine', 'United Kingdom', 'France', 'Turkey', 'Sweden',
       'Montenegro', 'Russia', 'Russia', 'Italy', 'North Macedonia',
       'Italy', 'Italy', 'Sweden', 'United Kingdom', 'Ukraine',
       'United Kingdom', 'Moldova', 'Albania', 'Germany', 'Norway',
       'United Kingdom', 'United Kingdom', 'Germany', 'Germany', 'Italy',
       'Germany', 'Bulgaria', 'Poland', 'United Kingdom', 'Germany',
       'Serbia', 'Germany', 'Poland', 'Germany', 'Sweden', 'Germany',
       'Ireland', 'Italy'

In [8]:
df_crime_index.describe

<bound method NDFrame.describe of             country           city  crime index
0    United Kingdom       Bradford        71.24
1    United Kingdom       Coventry        68.35
2            France         Nantes        65.70
3        Kazakhstan         Almaty        64.17
4             Italy        Catania        63.51
..              ...            ...          ...
165          Turkey      Eskisehir        18.86
166         Germany         Munich        18.66
167     Switzerland           Bern        17.94
168     Switzerland         Zurich        17.26
169           Spain  San Sebastian        14.86

[170 rows x 3 columns]>

Looks good! Now I have to group it by country and get the mean value of the crime index for this country. Simple.

In [34]:
new_crime_index = df_crime_index.groupby("country").mean()["crime index"].reset_index() 

/var/folders/vj/ht3c_9md4r78d2zr0h85nbhm0000gn/T/ipykernel_1473/3715088047.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  new_crime_index = df_crime_index.groupby("country").mean()["crime index"].reset_index()


Checking how many and which countries are in the dataset.

In [76]:
new_crime_index.country.nunique()

39

In [59]:
new_crime_index.country.values

array(['Albania', 'Austria', 'Belarus', 'Belgium', 'Bulgaria', 'Croatia',
       'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy',
       'Kazakhstan', 'Latvia', 'Lithuania', 'Luxembourg', 'Moldova',
       'Montenegro', 'Netherlands', 'North Macedonia', 'Norway', 'Poland',
       'Portugal', 'Romania', 'Russia', 'Serbia', 'Slovakia', 'Slovenia',
       'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine',
       'United Kingdom'], dtype=object)

While I'm at it, I can also round the numbers to one point after the decimal and change the name of the column.

In [44]:
new_crime_index = new_crime_index.round(1)

In [47]:
new_crime_index = new_crime_index.rename(columns = {"crime index":"crime_index"})

And now I can import it to our schema. That was all!

In [48]:
#push the table into the database
# Specify which table within your database you want to push your data to. Give your table an unambiguous name.
# Example: flights_sp for Sina's flights table, flights_groupname or similar
table_name = 'new_crime_index'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.

# Write records stored in a dataframe to SQL database
engine = get_engine()

if engine!=None:
    try:
        new_crime_index.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The new_crime_index table was imported successfully.
